This notebook is intended to demonstrate how vessel segmentation methods of ITKTubeTK can be applied to multi-channel MRI (MRA + T1, T2, etc).

In [1]:
import itk
from itk import TubeTK as ttk

from itkwidgets import view

import numpy as np

In [2]:
ImageType = itk.Image[itk.F, 3]

ReaderType = itk.ImageFileReader[ImageType]
ResampleType = ttk.ResampleImage[ImageType]

reader = ReaderType.New(FileName="data/mra-Brain.mha")
reader.Update()
im1 = reader.GetOutput()
print(im1.GetSpacing())

itkVectorD3 ([0.419922, 0.419922, 1.25])


In [3]:
res = ResampleType.New()
res.SetInput(im1)
res.SetMakeHighResIso(True)
res.Update()
im1iso = res.GetOutput()
print(im1iso.GetSpacing())

itkVectorD3 ([0.419922, 0.419922, 0.419922])


In [4]:
xp=[-3.9,-54.8,12.3]

vSeg = ttk.SegmentTubes[ImageType].New()
vSeg.SetInput( im1iso )
vSeg.SetVerbose( False )
vSeg.SetRadiusInObjectSpace( 0.33 )
vSeg.SetMinRoundness(0.2)
vSeg.SetDynamicScale(False)
tube = vSeg.ExtractTubeInObjectSpace( xp, 1 )
tubeG = vSeg.GetTubeGroup()

In [5]:
# Example of accessing data in a tube
p = list(map(lambda x: tube.GetPoint(x).GetPositionInObjectSpace(), range(tube.GetNumberOfPoints())))
r = list(map(lambda x: tube.GetPoint(x).GetRadiusInObjectSpace(), range(tube.GetNumberOfPoints())))

In [6]:
TubesToImageFilterType = ttk.ConvertTubesToImage[ImageType]
tubesToImageFilter = TubesToImageFilterType.New()
tubesToImageFilter.SetUseRadius(True)
tubesToImageFilter.SetTemplateImage(im1)
tubesToImageFilter.SetInput(tubeG)
tubesToImageFilter.Update()

outputImage = tubesToImageFilter.GetOutput()
view(outputImage)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageF3; proxy o…

In [7]:
TTKImageMathType = ttk.ImageMath[ImageType,ImageType]

imMath = TTKImageMathType.New(Input = outputImage)
imMath.AddImages(im1, 2048, 1)
combinedImage = imMath.GetOutput()
view(combinedImage)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageF3; proxy o…